#### pre processing

In [ ]:
# _____________ IMPORT
#region
from calendar import c
from ntpath import join
from re import A
from tkinter.tix import DirSelectBox
from zlib import DEF_BUF_SIZE
import pandas as pd
import seaborn as sns # added to OGD_HMM
import numpy as np
import matplotlib.pyplot as plt
import shutil
from glob import glob
from pathlib import Path
# from os import path
import os
# import glob
from IPython.display import display
import math

#### action analysis

In [ ]:
ogd_data = pd.read_csv("Data\gaze_input_tobii_and_ogd\group_difficult\participant01_trial01_ogd.txt", sep='\t')
# drop columns with any NaN values (don't know why they occur sometimes?)
ogd_data.dropna(inplace=True)
ogd_data.reset_index(drop=True, inplace=True) 
            
# initialize the action df
df_actions = pd.DataFrame()
df_actions['Action'] = ogd_data['Action']
df_actions['start_time'] = ogd_data['start_time']
df_actions['end_time'] = ogd_data['end_time']

# add column with action as numbers to simplify finding changes
all_actions = ogd_data['Action'].unique().tolist()
dict_actions = {k: v for v, k in enumerate(all_actions)}
df_actions['Action Numbers'] = (df_actions['Action']).map(dict_actions)

In [ ]:
# create a variable called 'change' that tells whether the event changed (calculates the difference to previous row)
df_actions['change'] = df_actions['Action Numbers'].diff()
# set change to 0 in first column
#df_actions['change'][0] = 1

In [ ]:
# remove rows where the event did not change:
df_actions = df_actions.loc[df_actions['change'] !=0 ,:]


In [ ]:
# assign continous start and end times -> end_time of action 1 = start_time of action 2
# to prevent gaps for further analysis of fixations and saccades
df_actions['start_time_action'] = float(0*len(df_actions))
if df_actions['change'][32] != 0:
    df_actions['start_time_action'][32] = df_actions['end_time'][32-1]

In [ ]:
# add start time of action
df_actions['end_time_action'] = df_actions['start_time'].shift(-1)
df_actions['start_time_action'] = df_actions['start_time']

# add time of last action
df_actions['end_time_action'].iloc[len(df_actions)-1] = ogd_data['end_time'].iloc[len(ogd_data)-1]
df_actions


In [ ]:

# fix first column (starts when very first action in first fixation starts)
df_actions['start_time_action'][0] = ogd_data['start_time'][0]

In [ ]:
df_actions['start_time_event'] = df_actions['end_time'].shift(1)
df_actions['end_time_event'] = df_actions['end_time']

#### all in one

In [44]:
import pandas as pd

ogd_data = pd.read_csv("Data\gaze_input_tobii_and_ogd\group_difficult\participant01_trial01_ogd.txt", sep='\t')
fixationdata = pd.read_csv("Data\gaze_input_tobii_and_ogd\group_difficult\participant01_trial01_fixations.txt", sep='\t')

# drop columns with any NaN values (don't know why they occur sometimes?)
ogd_data.dropna(inplace=True)
ogd_data.reset_index(drop=True, inplace=True) 

# rename last col to 'action'
ogd_data.rename(columns={ogd_data.columns[-1]: 'action'}, inplace=True)

# initialize the action df
df_actions = pd.DataFrame()
df_actions['Action'] = ogd_data['action']
df_actions['start_time'] = ogd_data['start_time']
df_actions['end_time'] = ogd_data['end_time']

# add column with action as numbers to simplify finding changes
all_actions = ogd_data['action'].unique().tolist()
dict_actions = {k: v for v, k in enumerate(all_actions)}
df_actions['Action Numbers'] = (df_actions['Action']).map(dict_actions)


# create a variable called 'change' that tells whether the event changed (calculates the difference to previous row)
df_actions['change'] = df_actions['Action Numbers'].diff()

# remove rows where the event did not change:
df_actions = df_actions.loc[df_actions['change'] !=0 ,:]

# 
df_actions['start_time_action'] = df_actions['start_time']
df_actions['end_time_action'] = df_actions['start_time'].shift(-1)


# add time of first and last action
# end of trial time
end_time_seconds = float(fixationdata['Event End Trial Time [ms]'].iloc[len(fixationdata)-1]/1000)
df_actions['end_time_action'].iloc[len(df_actions)-1] = end_time_seconds
# start of trial time
df_actions['start_time_action'][0] = float(0)
df_actions


c:\Users\renee\anaconda3\envs\OGD_HMM\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\Users\renee\anaconda3\envs\OGD_HMM\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Action,start_time,end_time,Action Numbers,change,start_time_action,end_time_action
0,Cap Off,0.850,0.950,0,NaN,0.000,4.787
32,Apply Tip,4.787,4.887,1,1.0,4.787,13.232
109,Setting Units,13.232,13.302,2,1.0,13.232,15.981
133,Priming,15.981,16.081,3,1.0,15.981,20.438
173,Setting Units,20.438,20.508,2,-1.0,20.438,21.697
182,Priming,21.697,21.797,3,1.0,21.697,29.623
246,Setting Units,29.623,29.723,2,-1.0,29.623,32.801
274,Injection,32.801,32.901,4,2.0,32.801,53.379
455,Remove Tip,53.379,53.479,5,1.0,53.379,61.235
522,Cap On,61.235,61.335,6,1.0,61.235,69.200


In [42]:
df_actions.reset_index(inplace=True)
df_actions = df_actions.rename(columns = {'index':'action_change_index'})

In [43]:
df_actions

,Items,Action,start_time,end_time,Action Numbers,change,start_time_action,end_time_action
0,0,Cap Off,0.850,0.950,0,NaN,0.000,4.787
1,32,Apply Tip,4.787,4.887,1,1.0,4.787,13.232
2,109,Setting Units,13.232,13.302,2,1.0,13.232,15.981
3,133,Priming,15.981,16.081,3,1.0,15.981,20.438
4,173,Setting Units,20.438,20.508,2,-1.0,20.438,21.697
5,182,Priming,21.697,21.797,3,1.0,21.697,29.623
6,246,Setting Units,29.623,29.723,2,-1.0,29.623,32.801
7,274,Injection,32.801,32.901,4,2.0,32.801,53.379
8,455,Remove Tip,53.379,53.479,5,1.0,53.379,61.235
9,522,Cap On,61.235,61.335,6,1.0,61.235,69.200


#### separate ogd_data into new dataframes and calculate ooi_metrics

In [40]:
# extract actions
all_actions = ogd_data['action'].unique().tolist()

In [41]:
all_actions

['Cap Off',
 'Apply Tip',
 'Setting Units',
 'Priming',
 'Injection',
 'Remove Tip',
 'Cap On']

In [37]:
df_new = ogd_data[ogd_data['action'] == 'Cap Off']

In [38]:
df_new

,start_time,end_time,App,Cap,Gauge,Pad,Pen,Safety,Tip,action
0,0.850,0.950,300.8,24.7,183.8,551.0,0.0,368.1,1600.0,Cap Off
1,0.950,1.050,302.4,30.0,182.9,552.0,0.0,380.7,1600.0,Cap Off
2,1.050,1.100,305.4,31.8,182.4,560.0,0.0,387.4,1600.0,Cap Off
3,1.119,1.219,279.7,65.8,151.1,533.0,0.0,431.7,1600.0,Cap Off
4,1.269,1.369,265.7,98.7,120.2,503.1,0.0,468.4,1600.0,Cap Off
5,1.369,1.469,266.6,90.7,126.5,506.9,0.0,460.4,1600.0,Cap Off
6,1.469,1.569,268.3,87.2,129.5,511.0,0.0,456.5,1600.0,Cap Off
7,1.599,1.699,268.8,92.2,123.0,507.7,0.0,457.9,1600.0,Cap Off
8,1.819,1.919,358.7,0.0,244.0,650.4,36.7,317.6,1600.0,Cap Off
9,1.969,2.069,375.7,0.0,248.7,619.1,45.0,348.9,1600.0,Cap Off


In [ ]:
for i in range(1,len(df_actions)):
    df_new = ogd_data[df_actions.index[i-1]:df_actions.index[i]]

In [ ]:
df_new

In [ ]:
cut_off_rows = []
i=0
for row in range(1,len(ogd_data)-1):
    print('row:', row)
    print('i:', i)
    if ogd_data['end_time'][row] > df_actions['end_time_action'][i]:
        cut_off_rows.append(row-1)
        print('i:', i)
        i=i+1


In [ ]:
cut_off_rows

In [ ]:
ogd_data.iloc[0:5]